In [ ]:
import pretrainedmnist

net = pretrainedmnist.mnist(pretrained=True)

In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import transforms, datasets


In [ ]:
transform = transforms.Compose(
    [transforms.ToPILImage(),
     transforms.Resize((28,28)),
#     transforms.Grayscale(),
     transforms.ToTensor()])
#,
 #    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
from scipy.misc import imread, imsave
import io
import sqlite3
def make_connection(p):
    return sqlite3.connect(p)
def preload_all_queries(conn, more=''):
    c = conn.cursor()
    c.execute('''SELECT id_a,student,* FROM capture_zone WHERE type=4 '''+more+''' ORDER BY student,id_a,id_b ASC''')
    res = {}
    for r in c.fetchall():
        k = r[1]
        if not (k in res):
            res[k] = []
        r = r[:-1] + (imread(io.BytesIO(r[-1])),)
        #import pytesseract
        #print(pytesseract.image_to_string(imread(io.BytesIO(r[-1])), config='--psm 10'))
        # ^ doesn't seem to work too well on this data
        #import base64
        #r = r[:-1] + (base64.b64encode(r[-1]).decode('ascii'),)
        #im = imread(io.BytesIO(r[-1]))
        res[k].append(r)
    return res


In [ ]:
info = preload_all_queries(make_connection('test3/capture.sqlite'), more=' AND student='+str(42))

In [ ]:
from matplotlib import pyplot as plt
#im = info[42][2][13]
#im = np.transpose(info[42][2][13], [0,1,2])
#im = np.array([np.array(info[42][i][13]) for i in range(10)])
im = np.array([transform(info[42][i][13]).numpy()[0,:,:] for i in range(20)]) 
#print(im.shape)
#im = transform(im)
#plt.imshow(im[0,:,:])
#print(im.min(), im.max(), im.shape)
#im = (im - np.mean(im)) / np.std(im)

#transform(im.shape)
with torch.no_grad():
    print(im.size)
    pred = net(torch.from_numpy(im))
    print(pred.shape)
    print(np.argmax(pred, axis=1), pred)
    


In [ ]:
transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((28,28)),
        #transforms.Resize((50,50)),
        #transforms.CenterCrop((28,28)),
        #transforms.Grayscale(),
        transforms.ToTensor()
        ])


In [ ]:
im = info[42][81][13]
im0 = im
print(im.shape)
im = np.transpose(transform(im0).numpy(), [1,2,0])
print(im.shape)
R = im[:,:,0]
G = im[:,:,1]
B = im[:,:,2]
im = np.sum(im, axis=2) / 3 #, np.array(im[:,:,1:2])# - np.sum(im, axis=2, keepdims=True))

im[((B*2-G-R)>.05)] = np.max(im)
im[((R*2-G-B)>.05)] = np.max(im)

print(im.shape)
#plt.imshow(im)
plt.imshow(im)
#im = transform(im[:,:,0]).numpy()
